In [ ]:
import exp_config
from data_helper import *
import accumulators
from tqdm import tqdm

In [ ]:
mean_spectrum=accumulators.Mean()

for shot in tqdm(Run(exp_config.detector_keys,exp_config.database_keys, run=-20)):
    # do something with the data.
    spectrum = np.mean(shot.spectrometer, axis=1)
    mean_spectrum.accumulate(spectrum)
                                    
        
        
    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(mean_spectrum.value)